In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

import numpy as np
from collections import Counter

import pandas as pd






def most_frequent(List):
    return max(set(List), key = List.count)

def knn_model_train(X_train, y_train, neighbors): 
    knn_models = [None]*100
    for i in range(100):
        clf = KNeighborsClassifier(n_neighbors=neighbors)
        local_X_train = X_train[X_train.time==i+1].copy().drop(columns = ["time"])
        local_Y_train = Y_train[Y_train.time==i+1].setup
        knn_models[i] = clf.fit(local_X_train, local_Y_train.values.ravel())
    return knn_models

def DT_model_train(X_train, y_train): 
    DT_models = [None]*100
    
    for i in range(100):
        clf = tree.DecisionTreeClassifier()
        local_X_train = X_train[X_train.time==i+1].copy().drop(columns = ["time"])
        local_Y_train = Y_train[Y_train.time==i+1].setup
        DT_models[i] = clf.fit(local_X_train, local_Y_train.values.ravel())
    return DT_models

def model_test(X_test, y_test, models):
    prediction = []
    for j in range(160):
        prediction_vote = []
        for i in range(100):
            m = i
            i += j*100
            y_hat = models[m].predict(X_test[i:i+1]).item(0)
            prediction_vote.append(y_hat)
        prediction.append(most_frequent(prediction_vote))
    facit = y_test["setup"][::100]
    return facit, prediction    






df = pd.read_csv('armdatadf.csv')
#df['setup'] = df['setup'].apply(str)
df['setup'] = df['setup'].astype(int)

Y = df[["setup","person","time"]]
X = df[["x","y","z","person", "time"]]

facit = Y["setup"][::100].values

rightWrong_KNN_all = np.zeros((10,10*16))
rightWrong_KNN_noX = np.zeros((10,10*16))
rightWrong_KNN_noY = np.zeros((10,10*16))
rightWrong_KNN_noZ = np.zeros((10,10*16))

rightWrong_DT_all = np.zeros((10,10*16))
rightWrong_DT_noX = np.zeros((10,10*16))
rightWrong_DT_noY = np.zeros((10,10*16))
rightWrong_DT_noZ = np.zeros((10,10*16))

Gen_error_KNN_all = 0
Gen_error_KNN_noX = 0
Gen_error_KNN_noY = 0
Gen_error_KNN_noZ = 0

Gen_error_DT_all = 0
Gen_error_DT_noX = 0
Gen_error_DT_noY = 0
Gen_error_DT_noZ = 0

n_persons = 10
nn = 1

for i in range(1,n_persons+1):
    #Seperation
    X_train = X[(X.person != i)].copy()
    X_train['x'] = (X_train['x']-X_train['x'].mean())/X_train['x'].std()
    X_train['y'] = (X_train['y']-X_train['y'].mean())/X_train['y'].std()
    X_train['z'] = (X_train['z']-X_train['z'].mean())/X_train['z'].std()

    X_test  = X[(X.person == i)].copy()
    X_test['x'] = (X_test['x']-X_test['x'].mean())/X_test['x'].std()
    X_test['y'] = (X_test['y']-X_test['y'].mean())/X_test['y'].std()
    X_test['z'] = (X_test['z']-X_test['z'].mean())/X_test['z'].std()

    Y_train = Y[(Y.person != i)].copy()
    Y_test  = Y[(Y.person == i)].copy()

    ##################################################################################################################
    #Training site with all 
    knn_models = knn_model_train(X_train[["x","y","z","time"]],Y_train,nn)
    DT_models = DT_model_train(X_train[["x","y","z","time"]],Y_train)

    # Testing site with all
    facit, prediction = model_test(X_test[["x","y","z"]], Y_test, knn_models)
    rightWrong_KNN_all[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_KNN_all += np.mean(facit==prediction)
    print("KNN: Completed person {} with all coordinates with an accuracy of {}".format(i, acc))

    facit, prediction = model_test(X_test[["x","y","z"]], Y_test, DT_models)
    rightWrong_DT_all[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_DT_all += np.mean(facit==prediction)
    print("DT: Completed person {} with all coordinates with an accuracy of {}".format(i, acc))

    ##################################################################################################################
    #Training site with no X 
    knn_models = knn_model_train(X_train[["y","z","time"]],Y_train,nn)
    DT_models = DT_model_train(X_train[["y","z","time"]],Y_train)

    # Testing site with no X
    facit, prediction = model_test(X_test[["y","z"]], Y_test, knn_models)
    rightWrong_KNN_noX[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_KNN_noX += np.mean(facit==prediction)
    print("KNN: Completed person {} with no X with an accuracy of {}".format(i, acc))

    facit, prediction = model_test(X_test[["y","z"]], Y_test, DT_models)
    rightWrong_DT_noX[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_DT_noX += np.mean(facit==prediction)

    print("DT: Completed person {} with no X with an accuracy of {}".format(i, acc))

    ##################################################################################################################
    #Training site with no Z 
    knn_models = knn_model_train(X_train[["y","x","time"]],Y_train,nn)
    DT_models = DT_model_train(X_train[["y","x","time"]],Y_train)

    # Testing site with no Z
    facit, prediction = model_test(X_test[["y","x"]], Y_test, knn_models)
    rightWrong_KNN_noZ[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_KNN_noZ += np.mean(facit==prediction)
    print("KNN: Completed person {} with no Z with an accuracy of {}".format(i, acc))

    facit, prediction = model_test(X_test[["x","y"]], Y_test, DT_models)
    rightWrong_DT_noZ[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_DT_noZ += np.mean(facit==prediction)

    print("DT: Completed person {} with no Z with an accuracy of {}".format(i, acc))

    ##################################################################################################################
    #Training site with no Y 
    knn_models = knn_model_train(X_train[["z","x","time"]],Y_train,nn)
    DT_models = DT_model_train(X_train[["z","x","time"]],Y_train)

    # Testing site with no Y
    facit, prediction = model_test(X_test[["z","x"]], Y_test, knn_models)
    rightWrong_KNN_noY[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_KNN_noY += np.mean(facit==prediction)
    print("KNN: Completed person {} with no Y with an accuracy of {}".format(i, acc))

    facit, prediction = model_test(X_test[["x","z"]], Y_test, DT_models)
    rightWrong_DT_noY[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_DT_noY += np.mean(facit==prediction)

    print("DT: Completed person {} with no Y with an accuracy of {}".format(i, acc))

Gen_error_est_knn_noX = 1-(Gen_error_knn_noX / n_persons)
Gen_error_est_knn_noY = 1-(Gen_error_knn_noY / n_persons)
Gen_error_est_knn_noZ = 1-(Gen_error_knn_noZ / n_persons)
Gen_error_est_knn_all = 1-(Gen_error_knn_all / n_persons)

Gen_error_est_DT_noX  = 1-(Gen_error_DT_noX / n_persons)
Gen_error_est_DT_noY  = 1-(Gen_error_DT_noY / n_persons)
Gen_error_est_DT_noZ  = 1-(Gen_error_DT_noZ / n_persons)
Gen_error_est_DT_all  = 1-(Gen_error_DT_all / n_persons)

print("Completed {} number of neighbours with all coordinates with an estimated generalization error of {}.".format(nn, Gen_error_est_knn_all))
print("Completed {} number of neighbours with no X with an estimated generalization error of {}.".format(nn, Gen_error_est_knn_noX))
print("Completed {} number of neighbours with no Y with an estimated generalization error of {}.".format(nn, Gen_error_est_knn_noY))
print("Completed {} number of neighbours with no Z with an estimated generalization error of {}.".format(nn, Gen_error_est_knn_noZ))

print("Completed Decision Tree with all coordinates an estimated generalization error of {}.".format(Gen_error_est_DT_all))
print("Completed Decision Tree with no X an estimated generalization error of {}.".format(Gen_error_est_DT_noX))
print("Completed Decision Tree with no Y an estimated generalization error of {}.".format(Gen_error_est_DT_noY))
print("Completed Decision Tree with no Z an estimated generalization error of {}.".format(Gen_error_est_DT_noZ))

#the returned generalization error when testing   (number of correctly classified setups in test)/(length of trajectories in test)

In [ ]:
df = pd.read_csv('armdatadf.csv')
#df['setup'] = df['setup'].apply(str)
df['setup'] = df['setup'].astype(int)

Y = df[["setup","person","time"]]
X = df[["x","y","z","person", "time"]]

facit = Y["setup"][::100].values


df = pd.read_csv('armdatadf.csv')
df['setup'] = df['setup'].astype(int)

X = df[["x","y","z"]]
Y = df["setup"]

X.x = (X.x-X.x.mean()) / X.x.std()
X.y = (X.y-X.y.mean()) / X.y.std()
X.z = (X.z-X.z.mean()) / X.z.std()

# XM = np.empty([300])

# for i in range(1600):
#     XM = np.vstack([XM,X[["x","y","z"]][i*100:(i+1)*100].values.ravel()])

# XM = np.delete(XM,0,0)

# clf = KNeighborsClassifier(n_neighbors=1)
# XM
# preds = []
# generr = 0
# for i in range(10):
#     clf = KNeighborsClassifier(n_neighbors=100)
#     test_IDS = list(range(160*i,160*(i+1)))
#     X_test   = XM[test_IDS]
#     X_train  = np.delete(XM, test_IDS, axis = 0)
    
#     y_test = facit[test_IDS]
#     y_train  = np.delete(facit,test_IDS)

#     #Training site
#     model = clf.fit(X_train,y_train)

#     #Testing site
#     preds.append(model.predict(X_test))
#     generr += np.mean(preds == y_test)
# generr /= 10
# print(generr)
#the returned generalization error 

/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
0.2646416170634921


In [ ]:
rightWrong_KNN_all = np.zeros((10,10*16))
rightWrong_KNN_noX = np.zeros((10,10*16))
rightWrong_KNN_noY = np.zeros((10,10*16))
rightWrong_KNN_noZ = np.zeros((10,10*16))

rightWrong_DT_all = np.zeros((10,10*16))
rightWrong_DT_noX = np.zeros((10,10*16))
rightWrong_DT_noY = np.zeros((10,10*16))
rightWrong_DT_noZ = np.zeros((10,10*16))


df = pd.DataFrame (rightWrong_KNN_all)
filepath = 'rightWrong_KNN_all.xlsx'
df.to_excel(filepath, index=False)

df = pd.DataFrame (rightWrong_KNN_noX)
filepath = 'rightWrong_KNN_noX.xlsx'
df.to_excel(filepath, index=False)

df = pd.DataFrame (rightWrong_KNN_noY)
filepath = 'rightWrong_KNN_noY.xlsx'
df.to_excel(filepath, index=False)

df = pd.DataFrame (rightWrong_KNN_noZ)
filepath = 'rightWrong_KNN_noZ.xlsx'
df.to_excel(filepath, index=False)

df = pd.DataFrame (rightWrong_KNN_noX)
filepath = 'rightWrong_KNN_noX.xlsx'
df.to_excel(filepath, index=False)

df = pd.DataFrame (rightWrong_DT_all)
filepath = 'rightWrong_DT_all.xlsx'
df.to_excel(filepath, index=False)

df = pd.DataFrame (rightWrong_DT_noX)
filepath = 'rightWrong_DT_noX.xlsx'
df.to_excel(filepath, index=False)

df = pd.DataFrame (rightWrong_DT_noY)
filepath = 'rightWrong_DT_noY.xlsx'
df.to_excel(filepath, index=False)

df = pd.DataFrame (rightWrong_DT_noZ)
filepath = 'rightWrong_DT_noZ.xlsx'
df.to_excel(filepath, index=False)

In [ ]:
rightWrong = np.zeros((10,10*16))
Gen_error_est = []
n_persons = 10
# nn_to_try_out = [1,2,3,5,8,10,15,20]
nn_to_try_out = [1,2,3,5,8,10,15,20]
for nn in nn_to_try_out:
    Gen_error = 0
    for i in range(1,n_persons+1):
        #Seperation
        X_train = X[(X.person != i)].copy()
        X_train['x'] = (X_train['x']-X_train['x'].mean())/X_train['x'].std()
        X_train['y'] = (X_train['y']-X_train['y'].mean())/X_train['y'].std()
        X_train['z'] = (X_train['z']-X_train['z'].mean())/X_train['z'].std()

        X_test  = X[(X.person == i)].copy()
        X_test['x'] = (X_test['x']-X_test['x'].mean())/X_test['x'].std()
        X_test['y'] = (X_test['y']-X_test['y'].mean())/X_test['y'].std()
        X_test['z'] = (X_test['z']-X_test['z'].mean())/X_test['z'].std()

        Y_train = Y[(Y.person != i)].copy()
        Y_test  = Y[(Y.person == i)].copy()

        #Training site
        models = knn_model_train(X_train,Y_train,nn)

        #Testing site
        facit, prediction = knn_model_test(X_test, Y_test, models)
        rightWrong[i] = (facit==prediction)*1
        acc = np.mean(facit==prediction)
        Gen_error += np.mean(facit==prediction)
        print("Completed person {} with an accuracy of {}".format(i, acc))

    Gen_avr = Gen_error / n_persons
    Gen_error_est.append(Gen_avr)
    print("Completed {} number of neighbours with an estimated generalization accuracy of {}.".format(nn, Gen_avr))
print("The final generalization accuracies: ")
print(Gen_error_est)
#the returned generalization error when testing   (number of correctly classified setups in test)/(length of trajectories in test)

In [ ]:
[1,2,3,5,8,10,15,20]
[0.327, 0.264, 0.2647, 0.3016, 0.3244, 0.33, 0.3263, 0.3325]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9cabbf1f-ad60-47b3-af3c-cdf3732148b9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>